In [1]:
from neuroml import Morphology, Segment, Point3DWithDiam as P
from pyNN.morphology import NeuroMLMorphology

In [2]:
soma = Segment(proximal=P(x=0, y=0, z=0, diameter=18.8),
               distal=P(x=18.8, y=0, z=0, diameter=18.8),
               name="soma", id=0)
dend = Segment(proximal=P(x=0, y=0, z=0, diameter=2),
               distal=P(x=-500, y=0, z=0, diameter=2),
               name="dendrite",
               parent=soma, id=1)
neuroml_morph = Morphology(segments=(soma, dend))
pynn_morph = NeuroMLMorphology(neuroml_morph)

In [3]:
import arbor
import re

In [4]:
# methods already defined in PyNN_neuroml_morphology_IN_Arbor.ipynb
def create_arbor_tree(nml_morph):
    tree = arbor.segment_tree()
    for i, nml_seg in enumerate(nml_morph.segments):
        append_arbor_tree(tree, nml_seg)
    return tree
    
def append_arbor_tree(tree, nml_seg):
    if not nml_seg.parent:
        tree.append(arbor.mnpos,
                    arbor.mpoint(nml_seg.proximal.x, nml_seg.proximal.y, nml_seg.proximal.z,
                                 nml_seg.proximal.diameter/2),
                    arbor.mpoint(nml_seg.distal.x, nml_seg.distal.y, nml_seg.distal.z,
                                 nml_seg.distal.diameter/2), tag=get_swc_tag(nml_seg))
    else:
        tree.append(nml_seg.parent.id,
                    arbor.mpoint(nml_seg.proximal.x, nml_seg.proximal.y, nml_seg.proximal.z,
                                 nml_seg.proximal.diameter/2),
                    arbor.mpoint(nml_seg.distal.x, nml_seg.distal.y, nml_seg.distal.z,
                                 nml_seg.distal.diameter/2), tag=get_swc_tag(nml_seg))

def get_swc_tag(nml_seg):
    if re.search("soma", nml_seg.name, re.IGNORECASE):
        return 1
    elif re.search("axon", nml_seg.name, re.IGNORECASE):
        return 2
    elif re.search("dend", nml_seg.name, re.IGNORECASE):
        return 3
    else:
        return 5

In [5]:
morph_tree = create_arbor_tree(pynn_morph)

In [6]:
#Arbor uses a domains specific language (DSL) to describe regions and locations, which are given labels.
def create_region_definitions(pynn_morph):
    dict_defs = {}
    for i, nml_seg in enumerate(pynn_morph.segments):
        dict_defs.update({nml_seg.name: "(tag "+ str(get_swc_tag(nml_seg))+ ")"})
    return dict_defs

In [7]:
#dict_defs => {'soma': '(tag 1)', 'dendrite': '(tag 3)'}
morph_labels = arbor.label_dict(create_region_definitions(pynn_morph))

In [8]:
# Cell builders need to refer to regions and locations on a cell morphology.
cell = arbor.cable_cell(morph_tree, morph_labels)

In [9]:
print(cell.locations)

<bound method PyCapsule.locations of <arbor.cable_cell>>


In [10]:
create_region_definitions(pynn_morph)

{'soma': '(tag 1)', 'dendrite': '(tag 3)'}

In [11]:
help(cell.locations)

Help on method locations in module arbor._arbor:

locations(...) method of arbor._arbor.cable_cell instance
    locations(self: arbor._arbor.cable_cell, label: str) -> List[arb::mlocation]
    
    The locations of the cell morphology for a locset label.



In [12]:
print(cell.locations("dendrite"))

RuntimeError: no definition for 'dendrite'

In [12]:
import pyNN.neuron as sim

In [13]:
sim.NaChannel

pyNN.neuron.standardmodels.ion_channels.NaChannel

In [14]:
from pyNN.standardmodels import ion_channels as standard, build_translations

In [15]:
# arbor/standardmodels/ion_channels.py
class arbor_NaChannel(standard.NaChannel):
    """
    Exactly the same as neuron.NaChannel because
    https://arbor.readthedocs.io/en/pydoc/mechanisms.html?highlight=huxley#density-mechanisms
    https://arbor.readthedocs.io/en/pydoc/nmodl.html#nmodl
    """
    translations = build_translations(("conductance_density", "gnabar"), #(pynn_name, sim_name)
                                      ("e_rev", "ena"),)
    variable_translations = {"m": ("hh", "m"),
                             "h": ("hh", "h"),}
    model = "hh"
    conductance_density_parameter = "gnabar"
    
class arbor_KdrChannel(standard.KdrChannel):
    translations = build_translations(
        ('conductance_density', 'gkbar'),
        ('e_rev', 'ek'),
    )
    variable_translations = {
        'n': ('hh', 'n')
    }
    model = "hh"
    conductance_density_parameter = 'gkbar'


class arbor_PassiveLeak(standard.PassiveLeak):
    translations = build_translations(
        ('conductance_density', 'g'),
        ('e_rev', 'e'),
    )
    model = "pas"
    conductance_density_parameter = 'g'

In [16]:
build_translations(("conductance_density", "gnabar"), ("e_rev", "ena"),)

{'conductance_density': {'translated_name': 'gnabar',
  'forward_transform': 'conductance_density',
  'reverse_transform': 'gnabar'},
 'e_rev': {'translated_name': 'ena',
  'forward_transform': 'e_rev',
  'reverse_transform': 'ena'}}

In [17]:
from pyNN.parameters import IonicSpecies

In [18]:
help(IonicSpecies)

Help on class IonicSpecies in module pyNN.parameters:

class IonicSpecies(builtins.object)
 |  IonicSpecies(ion_name, reversal_potential, internal_concentration=None, external_concentration=None)
 |  
 |  Encapsulation of parameters for an ionic species
 |  
 |  Methods defined here:
 |  
 |  __init__(self, ion_name, reversal_potential, internal_concentration=None, external_concentration=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [19]:
ionic_species = {"na": IonicSpecies("na", reversal_potential=50.),
                 "k": IonicSpecies("k", reversal_potential=-77.),}

In [20]:
ionic_species

{'na': <pyNN.parameters.IonicSpecies at 0x7fab908021c0>,
 'k': <pyNN.parameters.IonicSpecies at 0x7fab90802550>}

In [ ]:
cell_type = cell_class(morphology=NeuroMLMorphology(Morphology(segments=(soma, dend))),  # yuck
                       cm=1.0,
                       Ra=500.0,
                       ionic_species={
                              "na": IonicSpecies("na", reversal_potential=50.0),
                              "k": IonicSpecies("k", reversal_potential=-77.0)
                       },
                       pas={"conductance_density": uniform('all', 0.0003),
                            "e_rev":-54.3},
                       na={"conductance_density": uniform('soma', 0.120),
                           "e_rev": 50.0},
                       kdr={"conductance_density": uniform('soma', 0.036),
                            "e_rev": -77.0}
                       )

In [25]:
# Use Nernst to calcultae reversal potential for Ca
#cell.set_ion("ca", method=arbor.mechanism("nernst/x=ca"))

In [21]:
help(arbor.mechanism.set)

Help on instancemethod in module arbor._arbor:

set(...)
    set(self: arbor._arbor.mechanism, name: str, value: float) -> None
    
    Set parameter value.



In [22]:
help(cell.set_ion)

Help on method set_ion in module arbor._arbor:

set_ion(...) method of arbor._arbor.cable_cell instance
    set_ion(self: arbor._arbor.cable_cell, ion: str, int_con: Optional[float] = None, ext_con: Optional[float] = None, rev_pot: Optional[float] = None, method: Optional[arb::mechanism_desc] = None) -> None
    
    Set the propoerties of ion species named 'ion' that will be applied
    by default everywhere on the cell. Species concentrations and reversal
    potential can be overridden on specific regions using the paint interface, 
    while the method for calculating reversal potential is global for all
    compartments in the cell, and can't be overriden locally.
     ion:     name of ion species.
     int_con: initial internal concentration [mM].
     ext_con: initial external concentration [mM].
     rev_pot: reversal potential [mV].
     method:  method for calculating reversal potential.



In [23]:
# Implementing
# ionic_species={"na": IonicSpecies("na", reversal_potential=50.0),
#                "k": IonicSpecies("k", reversal_potential=-77.0)}
cell.set_ion(ion="na", rev_pot=50.)
cell.set_ion(ion="k", rev_pot=-77.)
# Note in PyNN this is done for the whole cell but in Arbor this can be set for specific sections (like soma, axon, etc.)
#cell.paint("soma", arbor.ion("ca", int_con=2e-4, ext_con=2.5, rev_pot=114))
#cell.paint("axon", arbor.ion("ca", rev_pot=126))
#https://arbor.readthedocs.io/en/pydoc/cable_cell.html?highlight=reverse%20potential#ion-species

In [24]:
pas={"conductance_density": uniform('all', 0.0003),
                            "e_rev":-54.3},
                       na={"conductance_density": uniform('soma', 0.120),
                           "e_rev": 50.0},
                       kdr={"conductance_density": uniform('soma', 0.036),
                            "e_rev": -77.0}

Help on built-in function cable_probe in module arbor._arbor:

cable_probe(...) method of builtins.PyCapsule instance
    cable_probe(kind: str, id: arbor._arbor.cell_member, location: arbor._arbor.location) -> arb::probe_info
    
    Description of a probe at a location on a cable cell with id available for monitoring data of kind where kind is one of 'voltage' or 'ionic current density'.



In [25]:
from pyNN.morphology import uniform

In [28]:
k = uniform('all', 0.0003)

In [30]:
k.value

0.0003

Mechanism Info:
Meta data about the fields and ion dependencies of a mechanism. The data is presented as read-only attributes.

In [31]:
cat = arbor.default_catalogue()

In [32]:
print(cat['hh'].parameters['gnabar'])

{units: 'S / cm2', default: 0.12, min: -1.79769e+308, max: 1.79769e+308}


In [33]:
ions = cat['hh'].ions

In [34]:
print(ions.keys())

dict_keys(['k', 'na'])


Describe a mechanism that is to be painted or placed on the cable cell.

Mechanisms describe physical processes, distributed over the membrane of the cell. Density mechanisms are associated with regions of the cell, whose dynamics are a function of the cell state and their own state where they are present. Point mechanisms are defined at discrete locations on the cell, which receive events from the network. A third, specific type of density mechanism, which describes ionic reversal potential behaviour, can be specified for cells or the whole model.

In [35]:
hh = arbor.mechanism('hh') # w/ default values

In [36]:
pas = arbor.mechanism('pas', {'e': -55, 'gl': 0.02}) # custom values

In [ ]:
# Create pas mechanism with default parameter values (set in NOMDL file).
m1 = arbor.mechanism('passive')

# Create default mechainsm with custom conductance (range).
m2 = arbor.mechanism('passive', {'g', 0.1})

# Create a new pas mechanism with that changes reversal potential (global).
m3 = arbor.mechanism('passive/el=-45')

# Create an instance of the same mechanism, that also sets conductance (range).
m4 = arbor.mechanism('passive/el=-45', {'g', 0.1})

# This is an equivalent to m4, using set method to specify range parameters.
m5 = arbor.mechanism('passive/el=-45')
m5.set('g', 0.1)

# Decorate the 'soma' on a cable_cell.

cell.paint('soma', m1)
cell.paint('soma', m2) # Error: can't place the same mechanism on overlapping regions
cell.paint('soma', m3) # This would be ok: m3 is a new, derived mechanism by virtue of
                       # having a different name, i.e. 'passive/el=-45' vs. 'passive'.